In [1]:
# 1. Install dependencies
!pip install yt-dlp
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg -y


In [2]:
# 2. Import libraries
import whisper
import os
import yt_dlp


In [3]:
# 3. Define YouTube download function
def download_audio(youtube_url, output_path="audio.mp3"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': False,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])


In [4]:
# 4. Download the audio from YouTube
video_url = input("Paste your YouTube video link: ")
download_audio(video_url)


In [5]:
# 5. Load Whisper model
model = whisper.load_model("base")  # or use "small", "medium", "large"


In [6]:
# 6. Transcribe audio and export to SRT
result = model.transcribe("audio.mp3", task="transcribe", verbose=True)
with open("subtitles.srt", "w", encoding="utf-8") as srt_file:
    for i, segment in enumerate(result["segments"]):
        start = segment["start"]
        end = segment["end"]
        text = segment["text"].strip()

        def format_time(seconds):
            h = int(seconds // 3600)
            m = int((seconds % 3600) // 60)
            s = int(seconds % 60)
            ms = int((seconds - int(seconds)) * 1000)
            return f"{h:02}:{m:02}:{s:02},{ms:03}"

        srt_file.write(f"{i+1}\n{format_time(start)} --> {format_time(end)}\n{text}\n\n")

print("✅ Subtitle file generated: subtitles.srt")